- Image scale normalization ✅
- Image format normalization (.jpg, .png) ✅
- overlay ✅
- blurring ✅
- pixelate ✅
- sharpness 
- shifting ✅
- etching ✅
- flipping ✅
- flat shading ✅
- denoising
- motion blur
- bokeh (layered kernels)

In [1]:
# import dependencies

import matplotlib.image as mimg
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import itertools


In [2]:
# Image initialization
def rgba2rgb(img):
    height, width, _ = img.shape
    rgb_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                rgb_img_arr[i, j, k] = img.item((i, j, k))

    return rgb_img_arr


# 0.299R + 0.587G + 0.114B.
def rgb2gray(img):
    height, width, _ = img.shape
    gray_img_arr = np.zeros(shape=(height, width))

    for i in range(height):
        for j in range(width):
            r_gray = img.item((i, j, 0)) * 0.299
            g_gray = img.item((i, j, 1)) * 0.587
            b_gray = img.item((i, j, 2)) * 0.114

            gray_img_arr[i, j] = r_gray + g_gray + b_gray

    return gray_img_arr


def _get_image(path, ignore_alpha = True):
    img = mimg.imread(path)

    if path.endswith('.jpg') or path.endswith('.tif') or path.endswith('.jpeg'):
        return img / 255 
        
    elif path.endswith('.png') and ignore_alpha:
        return rgba2rgb(img)

    return img

img1 = _get_image('Input Images/Spiral.jpg')
img2 = _get_image('Input Images/Oogway_forest.jpg')
img3 = _get_image('Input Images/F.png')
# img4 = _get_image('Input Images/Portrait.jpg')
# img5 = _get_image('Input Images/Bokeh1.png')
# img6 = _get_image('Input Images/Bokeh2.png')
img7 = _get_image('Input Images/Doge.png')
img9 = _get_image('Input Images/Swirly.jpg')
img10 = _get_image('Input Images/Dark Space.png', ignore_alpha=False)
# img11 = _get_image('Input Images/BigTest.jpeg')

In [ ]:
# dependency functions

# overlay dependency
def _avg_pixel_weighted(channel_val1, weight1, channel_val2, weight2) -> int:
    adjusted_weight_total = weight1 + weight2
    return ( (channel_val1 * weight1 + channel_val2 * weight2)/adjusted_weight_total)

def normalise_range(img):
    max_val = np.max(img)
    min_val = np.min(img)

    factor = 255/(max_val - min_val)

    return img * factor



# stop width for pixelate
def _find_stop_pixelate(height,  width, grid_size, start_pixel) -> list:
    x_dim, y_dim = start_pixel
    padding = grid_size//2
    _stop = (padding + 1) if grid_size & 1 else padding

    e_h = height % grid_size if (height - x_dim) <= padding else None # second axis
    e_w = width % grid_size if (width - y_dim) <= padding else None # first axis

    empty_stop_height = -padding + e_h if e_h is not None else _stop
    empty_stop_width = -padding + e_w if e_w is not None else _stop      # refactor _stop to above

    return [empty_stop_height, empty_stop_width]

# stop width for blur
def _find_stop_blur(height, width, grid_size, start_pixel) -> list:
    x_dim, y_dim = start_pixel
    padding = grid_size//2
    _stop = (padding + 1) if grid_size & 1 else padding

    if x_dim < height or y_dim < width:
        e_h = height - x_dim if (height - x_dim) <= padding and (x_dim < height) else _stop
        e_w = width - y_dim if (width - y_dim) <= padding and (y_dim < width) else _stop

    if x_dim >= height or y_dim >= width:
        e_h = height - (x_dim - padding) if (height - x_dim) <= padding and (x_dim >= height) else _stop
        e_w = width - (y_dim - padding) if (width - y_dim) <= padding and (y_dim >= width) else _stop

    return [e_h, e_w]

# blur, pixelate dependency
def _avg_color_in_grid(img, grid_size, start_pixel, img_type):

    x_dim, y_dim = start_pixel
    padding = grid_size//2 # no. of pixels on either side of central pixel

    _stop = (padding + 1) if grid_size & 1 else padding  # (less on right side if even)

    r_total, g_total, b_total = 0, 0, 0

    # height, width, _ = img.shape

    # if x_dim < height - _stop and y_dim < width - _stop:
    try:
        r_total, g_total, b_total = 0, 0, 0

        # default grid_size^2 for all pixels not on top or left edge
        total_pixels = grid_size**2 if x_dim > padding and y_dim > padding else 0

        for _i in range(-padding, _stop):
            for _j in range(-padding, _stop):

                if (x_dim + _i >= 0 ) and (y_dim + _j >= 0):

                    r_total += img.item((x_dim + _i, y_dim + _j, 0))
                    g_total += img.item((x_dim + _i, y_dim + _j, 1))
                    b_total += img.item((x_dim + _i, y_dim + _j, 2))

                    # increment by one if pixels on top or left edge
                    total_pixels += 1 and total_pixels != grid_size**2

    # else:
    except IndexError:
        r_total, g_total, b_total = 0, 0, 0
        height, width, _ = img.shape

        if img_type == "BLUR":
            empty_stop_height, empty_stop_width = _find_stop_blur(height, width, grid_size, start_pixel)
        elif img_type == "PIXELATE":
            empty_stop_height, empty_stop_width = _find_stop_pixelate(height, width, grid_size, start_pixel)

        total_pixels = 0
        for _i in range(-padding, empty_stop_height):
            for _j in range(-padding, empty_stop_width):

                r_total += img.item((x_dim + _i, y_dim + _j, 0))
                g_total += img.item((x_dim + _i, y_dim + _j, 1))
                b_total += img.item((x_dim + _i, y_dim + _j, 2))

                total_pixels+=1

    return np.array((r_total, g_total, b_total)) / total_pixels

In [ ]:
# CUSTOM FUNCTIONS

def single_cfunc(func, img):
    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=((height, width, 3)))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                new_img_arr[i, j, k] = func(img.item((i, j, k)))
    
    return new_img_arr

def multi_cfunc(func, *args):

    min_x_size = min([img.shape[0] for img in args])
    min_y_size = min([img.shape[1] for img in args])

    new_img_arr = np.zeros(shape=(min_x_size, min_y_size, 3))

    for i in range(min_x_size):
        for j in range(min_y_size):
            for k in range(3):
                new_img_arr[i, j, k] = func([img.item((i, j, k)) for img in args])
    
    return new_img_arr


In [ ]:
# RESIZE

def scale_img2img(target_img, control_img):
    c_height, c_width, _ = control_img.shape
    t_height, t_width, _ = target_img.shape

    h_factor = c_height/t_height
    w_factor = c_width/t_width

    new_img_arr = np.zeros(shape=(c_height, c_width, 3))

    for i in range(c_height):
        for j in range(c_width):
            for k in range(3):
                new_img_arr[i , j, k] = target_img.item((min(t_height-1, round(i/h_factor)), min(t_width-1, round(j/w_factor)), k))
    
    return new_img_arr

def scale_img2res(target_img, resolution):
    c_width, c_height = resolution.strip().split("x")
    c_height, c_width = int(c_height), int(c_width)
    t_height, t_width, _ = target_img.shape

    h_factor = c_height/t_height
    w_factor = c_width/t_width

    new_img_arr = np.zeros(shape=(c_height, c_width, 3))

    for i in range(c_height):
        for j in range(c_width):
            for k in range(3):
                new_img_arr[i , j, k] = target_img.item((min(t_height-1, round(i/h_factor)), min(t_width-1, round(j/w_factor)), k))
    
    return new_img_arr

def scale_img2factor(target_img, factor):
    c_height, c_width, _ = target_img.shape
    t_height, t_width = int(c_height*factor), int(c_width*factor)

    new_img_arr = np.zeros(shape=(t_height, t_width, 3))

    for i in range(t_height):
        for j in range(t_width):
            for k in range(3):
                new_img_arr[i , j, k] = target_img.item((min(c_height-1, round(i/factor)), min(c_width-1, round(j/factor)), k))
    
    return new_img_arr

In [ ]:
# FLIP

def flip(img, axis):
    if not 0 < axis < 3:
        raise ValueError("axis has to be 1 (vertical) or 2 (horizontal) ")

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                if axis == 1:
                    new_img_arr[i, j, k] = img.item((i, -j, k))
                else:
                    new_img_arr[i, j, k] = img.item((-i, j, k))
    
    return new_img_arr

In [ ]:
# OVERLAY

def overlay(img1, img1_weight, img2, img2_weight):

    min_x_size = min(img1.shape[0], img2.shape[0])
    min_y_size = min(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(min_x_size, min_y_size, 3))

    for i in range(min_x_size):
        for j in range(min_y_size):
            for k in range(3):

                new_img_arr[i, j, k] = _avg_pixel_weighted(img1.item((i, j, k)), img1_weight, img2.item((i, j, k)), img2_weight)  # 4 sec

    return new_img_arr

# MULTIPLY

# for some reason, multiply does NOT go well with .jpg compression
def multiply(img1, img2):

    min_x_size = min(img1.shape[0], img2.shape[0])
    min_y_size = min(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(min_x_size, min_y_size, 3))

    for i in range(min_x_size):
        for j in range(min_y_size):
            for k in range(3):
                new_img_arr[i, j, k] = img1.item((i, j ,k)) * img2.item((i, j ,k)) 

    return new_img_arr


# DARKEN

def darken(img1, img2):
    min_x_size = min(img1.shape[0], img2.shape[0])
    min_y_size = min(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(min_x_size, min_y_size, 3))

    for i in range(min_x_size):
        for j in range(min_y_size):
            color1 = img1[i, j]
            color2 = img2[i, j] 

            luminosity1 =  0.299 * color1[0] + 0.587 * color1[1] + 0.114 * color1[2]
            luminosity2 =  0.299 * color2[0] + 0.587 * color2[1] + 0.114 * color2[2]

            new_img_arr[i, j] = img1[i, j] if luminosity1 < luminosity2 else img2[i, j] 

    return new_img_arr

# LIGHTEN

def lighten(img1, img2):
    min_x_size = min(img1.shape[0], img2.shape[0])
    min_y_size = min(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(min_x_size, min_y_size, 3))

    for i in range(min_x_size):
        for j in range(min_y_size):
            color1 = img1[i, j]
            color2 = img2[i, j] 

            luminosity1 =  0.299 * color1[0] + 0.587 * color1[1] + 0.114 * color1[2]
            luminosity2 =  0.299 * color2[0] + 0.587 * color2[1] + 0.114 * color2[2]

            new_img_arr[i, j] = img1[i, j] if luminosity1 > luminosity2 else img2[i, j] 

    return new_img_arr

# DIFFERENCE

def difference(img1, img2):

    min_x_size = min(img1.shape[0], img2.shape[0])
    min_y_size = min(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(min_x_size, min_y_size, 3))

    for i in range(min_x_size):
        for j in range(min_y_size):
            for k in range(3):
                new_img_arr[i, j, k] = abs(img1.item((i, j ,k)) - img2.item((i, j ,k)))

    return new_img_arr

# POWER

def power(img1, img2):

    min_x_size = min(img1.shape[0], img2.shape[0])
    min_y_size = min(img1.shape[1], img2.shape[1])

    new_img_arr = np.zeros(shape=(min_x_size, min_y_size, 3))

    for i in range(min_x_size):
        for j in range(min_y_size):
            for k in range(3):
                new_img_arr[i, j, k] = img1.item((i, j ,k)) ** img2.item((i, j ,k))

    return new_img_arr

In [ ]:
# PIXELATE

def pixelate(img, pixelate_amount):

    height, width, _ = img.shape

    padding = pixelate_amount//2

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(padding, height + padding, pixelate_amount):
        for j in range(padding, width + padding, pixelate_amount):

            # finding average color
            _stop = (padding + 1) if pixelate_amount & 1 else padding # in grid

            total_pixels = 0
            r_total = 0
            g_total = 0
            b_total = 0
            
            for _i in range(-padding, _stop):
                for _j in range(-padding, _stop):
                    
                    _x = i + _i
                    _y = j + _j

                    if 0<= _x < height and 0 <= _y < width: # and (_x, _y) != (i, j):
                        r_total += img.item((_x, _y, 0))
                        g_total += img.item((_x, _y, 1))
                        b_total += img.item((_x, _y, 2))
                        total_pixels += 1
            
            color = np.array((r_total, g_total, b_total)) / total_pixels

            # applying color to all pixels
            for _i in range(-padding, _stop):
                for _j in range(-padding, _stop):
                    
                    _x = i + _i
                    _y = j + _j

                    if 0<= _x < height and 0 <= _y < width: # and (_x, _y) != (i, j):
                            new_img_arr[_x, _y] = color

    return new_img_arr


In [ ]:
# PIXELATE SAVE
# saves a true resolution version of the image (normal piexlate just makes nxn boxes of same color)

def pixelate_save(img, pixelate_amount):

    height, width, _ = img.shape
    new_img_arr = np.zeros(shape=(height//pixelate_amount, width//pixelate_amount, 3))
    pixelated_img = pixelate(img, pixelate_amount)

    for i in range(height//pixelate_amount):
        for j in range(width//pixelate_amount):
            for k in range(3):
                new_img_arr[i, j, k] = pixelated_img.item(i*pixelate_amount, j*pixelate_amount, k)


    return new_img_arr


In [ ]:
# OLD BLUR

# Blurs a given 2D array using an NxN matrix
# time complexity = O(n^2)
# * Works
def old_blur(img, blur_size = 3) -> np.ndarray:
    # 5 - ~35s
    # 10 - 2m 23.3s

    padding = blur_size//2
    __stop = padding + 1 if blur_size & 1 else padding
    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):

            total_pixels = 0
            r_total = 0
            g_total = 0
            b_total = 0

            for _i in range(-padding, __stop):
                for _j in range(-padding, __stop):
                    
                    _x = i + _i
                    _y = j + _j

                    if height > _x >= 0 and width > _y >= 0: # and (_x, _y) != (i, j):
                        r_total += img.item((_x, _y, 0))
                        g_total += img.item((_x, _y, 1))
                        b_total += img.item((_x, _y, 2))
                        total_pixels += 1

            new_img_arr[i, j] = np.array((r_total, g_total, b_total)) / total_pixels

    return new_img_arr

In [6]:
#! BLUR
#! refactor

# do you pass over the already-passed-once image or do you do it twice separately and then overlay?

# Blurs a given 2D array using two matrices (1xN and Nx1) in two separate passes
# this works because i am using 'mean blur' which is a separable kernel
# time complexity = O(n)

# note: works slower for 3x3 matrices than old blur but speeds up significantly with higher blur sizes.
def blur(img, blur_size = 3) -> np.ndarray:
    # 5 - ~32.4s
    # 10 - 46.8s

    padding = blur_size//2
    __stop = padding + 1 if blur_size & 1 else padding
    height, width, _ = img.shape

    temp_img = np.zeros(shape=(height, width, 3))
    new_img_arr = np.zeros(shape=(height, width, 3))

    # im first doing a vertical pass then a horizontal pass

    for i in range(height):
        for j in range(width):

            r_total = 0
            g_total = 0
            b_total = 0
            total_pixels = 0

            for _i in range(-padding, __stop):

                _x = i + _i

                if height > _x >= 0: # and (_x, _y) != (i, j):
                    r_total += img.item((_x, j, 0))
                    g_total += img.item((_x, j, 1))
                    b_total += img.item((_x, j, 2))
                    total_pixels += 1

            temp_img[i, j] = np.array((r_total, g_total, b_total)) / total_pixels # cannot use blur_size instead because of edge cases

    for i in range(height):
        for j in range(width):

            r_total = 0
            g_total = 0
            b_total = 0
            total_pixels = 0

            for _j in range(-padding, __stop):

                _y = j + _j

                if width > _y >= 0: # and (_x, _y) != (i, j):
                    r_total += temp_img.item((i, _y, 0))
                    g_total += temp_img.item((i, _y, 1))
                    b_total += temp_img.item((i, _y, 2))
                    total_pixels += 1

            new_img_arr[i, j] = np.array((r_total, g_total, b_total)) / total_pixels 

    del temp_img

    return new_img_arr

In [ ]:
# SHIFT

def shift(img, operation, start_val_increment=1.6, shift_x=10, shift_y=10, start_offset=0, horizontal=False):
    # operation = int(input("what operation should be performed? (0:linear, 1:sin, 2:cos, 3:tan, 4:random)  "))

    start_val = 0

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    if operation != 4:

        multiplicant = 0 #!

        for i in range(height):
            for j in range(width):
                for k in range(3):

                    new_img_arr[i, j, k] = img.item((
                        (int(multiplicant_x * shift_x) + i + start_offset) % height,
                        (int(multiplicant_y * shift_y) + j + start_offset) % width,
                        k))


                if not horizontal: start_val+=start_val_increment
                multiplicant_x = start_val if operation == 0 else math.sin(start_val) if operation == 1 else math.cos(start_val) if operation == 2 else math.tan(start_val) if operation == 3 else 0            
            if horizontal: start_val+=start_val_increment # for sin or cos, min and max shift is 0 and 1.5707 (π/2)
    else:

        multiplicant_x = 0
        multiplicant_y = 0

        for i in range(height):
            for j in range(width):
                for k in range(3):
                    # if operation == 4:
                    #     multiplicant = random.random() * 2 - 1

                    new_img_arr[i, j, k] = img.item((
                        (int(multiplicant_x * shift_x) + i + start_offset) % height,
                        (int(multiplicant_y * shift_y) + j + start_offset) % width,
                        k))

                multiplicant_x = random.random() * 2 - 1
                multiplicant_y = random.random() * 2 - 1

    return new_img_arr


In [ ]:
# SHARPEN (SCRAP)
# new idea:
#   new_pixel.channel = pixel.channel + (fine/255) * pixel.channel

def sharpen(img, sharpen_amt):

    blurred_img = blur(img, blur_size=sharpen_amt)
    fine_img = abs(img - blurred_img)

    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))
    fine_imgx3 = np.zeros(shape=(height, width, 3))

    for i in range(height):
        for j in range(width):
            for k in range(3):
                current_element = fine_img.item((i, j, k))
                if current_element != 0:
                    fine_imgx3[i, j, k] = min(1, current_element * 2) # creates halos

    for i in range(height):
        for j in range(width):
            for k in range(3):
                img_element = img.item((i, j, k))
                fine_img_element = fine_imgx3.item((i, j, k))

                new_img_arr[i, j, k] = fine_img_element if img_element < fine_img_element else img_element
    return new_img_arr


    # return _impose_fine(img, fine_img, threshold)
    # return fine_img
    # return(img + 0)

In [ ]:
# ETCH

def etch(img, brightness_ratio = 1):
    height, width, depth = img.shape

    new_img_arr = np.zeros(shape=(height, width, 4))

    for i in range(height):
        for j in range(width):

            if depth > 2:
                # check for transparency
                if img.item((i, j, 3)) < 0.035:
                    continue

            # getting grayscale value 
            r_gray = img.item((i, j, 0)) * 0.299 * brightness_ratio
            g_gray = img.item((i, j, 1)) * 0.587 * brightness_ratio
            b_gray = img.item((i, j, 2)) * 0.114 * brightness_ratio

            luminosity = r_gray + g_gray + b_gray

            # if pixel is dark, make it black
            # else make it transparent
            if luminosity <= 0.5:
                new_img_arr[i, j, 0] = 0
                new_img_arr[i, j, 1] = 0
                new_img_arr[i, j, 2] = 0
                new_img_arr[i, j, 3] = 1
            else:
                new_img_arr[i, j, 3] = 0


    return new_img_arr

    

In [ ]:
# FLAT SHADE

def flat(img, CM = 15):

    height, width, _ = img.shape
    new_img_arr = np.zeros(shape=(height, width, 3))

    __div = CM//2

    def int_round(num):
        rem = num % CM
        return num - rem if rem <= __div else num + CM - rem 


    for i in range(height):
        for j in range(width):
            for k in range(3):
                new_img_arr[i, j, k] = int_round(int(img.item((i, j, k)) * 255)) / 255
    
    return np.clip(new_img_arr, 0, 1)



In [ ]:
# DENOISE (WORK IN PROGRESS)

def denoise(img, strength = 1, threshold = 0.04):
    
    height, width, _ = img.shape

    for i in range(height):
        for j in range(width):

            for _i in range(-strength): # ! Incomplete
                # * Somehow find a way to check if this pixel-chunk is drastically different from its surrounding pixels.

                # * To select the pixels to exclude after determining that this is a denoisable pixel:
                # TODO: Test with standard deviation of border pixels, exclude pixels outside that std-dev, take average of remaining pixels and assign it to this pixel
                pass
            


In [ ]:
# MOTION BLUR (WORK IN PROGRESS)

def motion_blur(img, point, strength): # point is (x, y)

    height, width, _ = img.shape

    new_img_arr = img

    origin_x = point[0]
    origin_y = point[1]
    
    origin_to_point = np.zeros(shape=(2))

    # iter_height = list(range(height))
    # iter_width = list(range(width))

    iter_height = range(height)
    iter_width = range(width)

    # iter_height.remove(point[0])
    # iter_width.remove(point[1])

    index = 0 

    for y in iter_height:
        for x in iter_width:
            try:
                origin_to_point = np.array( (x - point[0], y - point[1]) ) / math.sqrt((x - point[0])**2 + (y - point[1])**2)
                # if np.isnan(origin_to_point.any()):
                #     origin_to_point = np.zeros(shape=(2))
            except ZeroDivisionError:
                origin_to_point = np.zeros(shape=(2))
            
            if np.isnan(origin_to_point[0]):
                    origin_to_point = np.zeros(shape=(2))
            dest_point = (np.clip(x+origin_to_point[0] * strength, 0, width), np.clip(y+origin_to_point[1] * strength, 0, height))

            R = img.item((y, x, 0))
            G = img.item((y, x, 1))
            B = img.item((y, x, 2))

            # slope = (origin_x - x)/(origin_y - y) # rfom origin to (x, y)

            slope = (y-dest_point[1]) / (x-dest_point[0])

            try:
                for x_final in range(int((dest_point[0] - x))):
                    y_final = int(np.round(slope * x_final))

                    new_img_arr[y_final, x_final, 0] += img.item((y_final, x_final, 0))
                    new_img_arr[y_final, x_final, 1] += img.item((y_final, x_final, 1))
                    new_img_arr[y_final, x_final, 2] += img.item((y_final, x_final, 2))
            except Exception: # !!!! 
                raise KeyboardInterrupt
            index += 1
    
    maximum = np.max(new_img_arr)
    minimum = np.min(new_img_arr)

    # print(maximum, minimum)

    a=0
    for i in range(height):
        for j in range(width):
            for k in range(3):
                # new_img_arr[i, j, k] = (  maximum - new_img_arr.item((i, j, k))   ) / (   maximum - minimum   )
                if np.isnan(new_img_arr.item((i, j, k))):
                    a+=1
    print(a)

    return new_img_arr



plt.imshow(motion_blur(img3, (40, 40), 6))

In [ ]:
# EDGE

# lower edge_threshold => more edges
def edge(img, edge_threshold = 20, color = False, strict = True, edge_dist=1):
    height, width, _ = img.shape

    edge_threshold /= 255

    new_img_arr = np.zeros(shape=(height, width, 3))

    final_value = (1.0, 1.0, 1.0)

    for i in range(height):
        for j in range(width):
            
            current_pixel_R = img.item((i, j, 0))
            current_pixel_G = img.item((i, j, 1))
            current_pixel_B = img.item((i, j, 2))

            if color: final_value = np.array((current_pixel_R, current_pixel_G, current_pixel_B))

            for _i, _j in itertools.product(range(-edge_dist, edge_dist+1), range(-edge_dist, edge_dist+1)):
                _x = i+_i
                _y = j+_j

                if 0 <= _x < height and 0 <= _y < width:

                    npxl_R = img.item((_x, _y, 0))
                    npxl_G = img.item((_x, _y, 1))
                    npxl_B = img.item((_x, _y, 2))

                    if strict:
                        if abs(current_pixel_R - npxl_R) > edge_threshold and abs(current_pixel_G - npxl_G) > edge_threshold and abs(current_pixel_B - npxl_B) > edge_threshold:
                            new_img_arr[i, j] = final_value
                            break
                    elif abs(current_pixel_R - npxl_R) > edge_threshold or abs(current_pixel_G - npxl_G) > edge_threshold or abs(current_pixel_B - npxl_B) > edge_threshold:
                        new_img_arr[i, j] = final_value
                        break 


    return new_img_arr

In [ ]:
# ROTATE

def rotate(img, degree = 45):
    height, width, _ = img.shape

    rad = -degree * np.pi / 180 # adding is ACW by default, i want it to be CW

    CENTRE_X = height/2 # 0
    CENTRE_Y = width/2  # 0
    
    def new_pos(x, y):
        dist = np.sqrt( (x-CENTRE_X)**2 + (y-CENTRE_Y)**2 )

        obj_theta = np.arctan2( y-CENTRE_Y, x-CENTRE_X )
        target_theta = obj_theta + rad # for twist, (rad + inc/800000)

        return (dist*np.cos(target_theta), dist*np.sin(target_theta))
    
    c1 = new_pos(0, 0)
    c2 = new_pos(height, 0)
    c3 = new_pos(0, width)
    c4 = new_pos(height, width)

    maxX = max(c1[0], c2[0], c3[0], c4[0])
    maxY = max(c1[1], c2[1], c3[1], c4[1])
    minX = min(c1[0], c2[0], c3[0], c4[0])
    minY = min(c1[1], c2[1], c3[1], c4[1])

    NEW_IMG_HEIGHT = int(maxX-minX) +1
    NEW_IMG_WIDTH = int(maxY-minY)  +1

    new_img_arr = np.zeros(shape=(NEW_IMG_HEIGHT, NEW_IMG_WIDTH, 3))

    for i in range(height):
        for j in range(width):

            target_pix_add = new_pos(i, j)

            target_pix_posX = NEW_IMG_HEIGHT/2 + target_pix_add[0]
            target_pix_posY = NEW_IMG_WIDTH/2 + target_pix_add[1]

            new_img_arr[int(target_pix_posX), int(target_pix_posY)] = img[i, j]
    
    return new_img_arr

In [ ]:
# GLITCH 

def glitch(img, chr_amount = 20, chunks = 2, shift_amt = 50):
    height, width, _ = img.shape

    new_img_arr = np.zeros(shape=(height, width, 3))

    R_offset = [random.randint(-chr_amount, chr_amount), random.randint(-chr_amount, chr_amount)]
    G_offset = [random.randint(-chr_amount, chr_amount), random.randint(-chr_amount, chr_amount)]
    B_offset = [random.randint(-chr_amount, chr_amount), random.randint(-chr_amount, chr_amount)]

    color_offset = [R_offset, G_offset, B_offset]

    for i in range(height):
        for j in range(width):
            for k in range(3):
                new_img_arr[(i+color_offset[k][0]) % height, (j+color_offset[k][1]) % width, k] = img.item((i, j, k))
    
    temp = new_img_arr.copy()

    for _ in range(chunks):
        line_start = random.randint(0, height)
        line_height = random.randint(0, int(height/4))

        shift_offset = random.randint(-shift_amt, shift_amt)

        for i in range(line_height):
            for j in range(width):
                new_img_arr[(line_start+i) % height, j] = temp[(line_start+i) % height, (j+shift_offset) % width]

    
    return new_img_arr
    

In [ ]:
# plt.imsave("Output Images/Overlay/Overlay 3-1.jpg", overlay(img1, img2))
# plt.imsave("Output Images/Blur/Blur 20.png", blurred_img)
# plt.imsave("Output Images/Sharpen/Doge1.png", sharpen(img7, 15))
# plt.imsave("Output Images/Distort/scds.png", shift(img2, 2, shift_x=6, shift_y=6, start_val_increment=0.8)) # eyesore
# plt.imsave("Output Images/Distort/Random.jpg", shift(img2, 4))
# plt.imsave("Output Images/Pixelate/Pixelate 20.png", pixelate(img1, 20))
# plt.imsave("Output Images/Etch/Etch img1.png", etch(img1, brightness_ratio= 1), cmap='gray') # 1.73
# plt.imsave('Output Images/Resize/Magic Factor.png', scale_img2factor(img6, 0.69))
# plt.imsave("Output Images/Overlay/Multiply/Multiply o3.png", multiply(img2, img9)) 
# plt.imsave("Output Images/Overlay/Darken/Darken o3.jpg", darken(img2, img9))
# plt.imsave("Output Images/Overlay/Lighten/Lighten o3.jpg", lighten(img2, img9))
# plt.imsave("Output Images/Overlay/Difference/Difference 1.jpg", difference(img1, img2))
# plt.imsave("Output Images/Overlay/Power/Power 102.jpg", power(img10, img9))
# plt.imsave("Output Images/Flip/Flip3.jpg", flip(img1, 1))
# plt.imsave("Output Images/Edge/Edge20.jpg", edge(img2, 20))
# plt.imsave("Output Images/Edge/Edge20 Color.jpg", edge(img2, 20, color=True))
# plt.imsave('Blur to.png', blur(img2, 5))
# plt.imsave('Output Images/Flat/Flat 35 tire.png', flat(_get_image('Input Images/IMI.png'), CM=35))
# plt.imsave('Output Images/Rotate/60D.jpg', rotate(img2, 60))
# plt.imsave('Output Images/Glitch/Glitche.jpg', glitch(img2, chunks=4))


In [ ]:
# luminosity method, B&W but bright enough to be seen
# 0.299R + 0.587G + 0.114B.

# denoising -> threshold (similar to cost function) passes threshold ? leave it be : make it its neighbouring color
# 
# completely isolated? make it its neighbouring color 

In [ ]:
# image1 = _get_image('../../Own/Input Images/1.jpeg') # ! - -
# image2 = _get_image('../../Own/Input Images/2.jpeg') # ! - -
# image3 = _get_image('../../Own/Input Images/3.jpeg') # ! - -
# image4 = _get_image('../../Own/Input Images/4.jpeg') # ! - -
# image5 = _get_image('../../Own/Input Images/5.jpeg') # ! - -
# image6 = _get_image('../../Own/Input Images/6.jpeg') # ! - -
# image7 = _get_image('../../Own/Input Images/7.jpeg') # ! - -
# image8 = _get_image('../../Own/Input Images/8.jpeg') # ! - -
# image9 = _get_image('../../Own/Input Images/9.jpeg') # ! - -
# image10 = _get_image('../../Own/Input Images/10.jpeg') # - -
# image11 = _get_image('../../Own/Input Images/11.jpeg') # - -

In [ ]:
# plt.imsave("../../Own/Output Images/portrait1.jpg", multiply(img4, scale_img2factor(image1, 1.71555555))) 
# plt.imsave("../../Own/Output Images/portrait2.jpg", multiply(img4, scale_img2factor(image2, 2.57333))) 
# plt.imsave("../../Own/Output Images/portrait3.jpg", multiply(img4, scale_img2factor(image3, 2.57333))) 
# plt.imsave("../../Own/Output Images/portrait4.jpg", multiply(img4, scale_img2factor(image4, 7.5317))) 
# plt.imsave("../../Own/Output Images/portrait5.jpg", multiply(img4, scale_img2factor(image5, 2.9))) 
# plt.imsave("../../Own/Output Images/portrait6.jpg", multiply(img4, scale_img2factor(image6, 2.5777777))) 
# plt.imsave("../../Own/Output Images/portrait7.jpg", multiply(img4, scale_img2factor(image7, 3.22222))) 
# plt.imsave("../../Own/Output Images/portrait8.jpg", multiply(img4, scale_img2factor(image8, 1.86795))) 
# plt.imsave("../../Own/Output Images/portrait9.jpg", multiply(img4, scale_img2factor(image9, 1.71555555))) 
# plt.imsave("../../Own/Output Images/portrait10.jpg", multiply(img4, scale_img2factor(image10, 2.54945))) 
# plt.imsave("../../Own/Output Images/portrait11.jpg", multiply(img4, scale_img2factor(image11, 1.93))) 
# plt.imsave("../../Own/Output Images/portrait edge.jpg", edge(img4))

In [7]:
plt.imsave("Blur 10 b.png", blur(img2, blur_size=10))

In [ ]:
# CIRCULAR

def max_difference(start_coord, end_coord):
    return max(abs(start_coord[0] - end_coord[0]), abs(start_coord[1] - end_coord[1]))


def lerp(start_coord, end_coord):
    x1 = start_coord[0]
    y1 = start_coord[1]
    x2 = end_coord[0]
    y2 = end_coord[1]

    x_len = x2-x1
    y_len = y2-y1

    max_diff = max(abs(x_len), abs(y_len))

    for m in range(max_diff):
        yield (x1 + int(m*x_len/max_diff), y1 + int(m*y_len/max_diff))

    '''

for i in range(1, 2):
    x=lerp((0, 0), (i*7, i*10))

    for _ in range(20): # TODO here DONT use max_diff+1, use maxdiff
        try:
            print(next(x))
        except StopIteration:
            pass
    '''

#* cap subtraction at zero

def circular(img):
    height, width, _ = img.shape

    CENTRE_X = int(height/2) # 0
    CENTRE_Y = int(width/2)  # 0

    coord_list = [(CENTRE_X+1, 0), (0, 0), (0, width), (height, width), (height, 0), (CENTRE_X, 0)]

    def update_coord_list(c_list):
        c_list[0] = (min(CENTRE_X, c_list[0][0]+1), 0)
        c_list[1] = (min(CENTRE_X, c_list[1][0] + 1), min(CENTRE_Y, c_list[1][1] + 1))
        c_list[2] = (min(CENTRE_X, c_list[2][0] + 1), max(CENTRE_Y, c_list[2][1] - 1))
        c_list[3] = (max(CENTRE_X, c_list[3][0] - 1), max(CENTRE_Y, c_list[3][1] - 1))
        c_list[4] = (max(CENTRE_X, c_list[4][0] - 1), min(CENTRE_Y, c_list[4][1] + 1))
        c_list[5] = c_list[0]

        return c_list

    radius_cor = (height, width)

    diag = int(np.sqrt(radius_cor[0]**2 + radius_cor[1]**2)) # max radius

    new_img_arr = np.zeros(shape=(diag, diag, 3))

    for _ in range(max_difference((0, 0), (CENTRE_X, CENTRE_Y))):
        
        for q in range(len(coord_list)-2):
            for _ in range(max_difference(coord_list[q], coord_list[q+1])):
                x = lerp(coord_list[q], coord_list[q+1])

                point = next(x)
                new_img_arr[point[0], point[1]] = img[point[0]-1, point[1]-1]
            
        coord_list = update_coord_list(coord_list)



            # target_pix_posX = diag/2 + radius*np.cos(target_theta)+200
            # target_pix_posY = diag/2 + radius*np.sin(target_theta)+200

            # new_img_arr[int(target_pix_posX), int(target_pix_posY)] = img[i, j]

    return new_img_arr

In [ ]:
plt.imsave('test.jpg', circular(img2))

In [ ]:
def gen_test(f):
    while True:
        yield f
        f+=2

x=gen_test(5)

print(next(x))
print(next(x))
print(next(x))
print(next(x))

x=gen_test(5)

print(next(x))
print(next(x))
print(next(x))
print(next(x))